In [27]:
import pyodbc 
import textwrap
import pandas as pd
import numpy as np
import json
from datetime import datetime
from datetime import date
import time
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from tabulate import tabulate
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.svm import SVR
from graphviz import Digraph
import graphviz
from sklearn import svm
from sklearn.ensemble import GradientBoostingRegressor
import pickle

In [28]:
#my driver
driver = '{ODBC Driver 17 for SQL Server}'

#specify name and database
serverName = "dublinbus-team12-server"
myDatabase = "dublinbus-team12-db"

#create server
server = "dublinbus-team12-server.database.windows.net"

#myport = 1433

#server = '{}.database.windows.net,{}'.format(serverName,myport)
myUsername = "innovationgeeks"
myPassword = "Laurawillsaveus445!"

#cnxn = pyodbc.connect('DRIVER={Devart ODBC Driver for SQLAzure};Server=server;Database=myDatabase;Port=myport;User ID=userid;Password=mypassword')

connection_string = textwrap.dedent('''
    Driver={driver};
    Server={server};
    Database={database};
    Uid={username};
    Pwd={password};
    Encrypt=yes;
    TrestServerCertificate=no;
    Connection Timeout=30
'''.format(driver=driver,server=server,database=myDatabase,username=myUsername,password=myPassword))

cnxn: pyodbc.Connection = pyodbc.connect(connection_string)
#cnxn = pyodbc.connect('DRIVER={Devart ODBC Driver for SQLAzure};Server=server;Database=myDatabase;Port=myport;User ID=userid;Password=mypassword')
#crsr: pyodbc.Cursor = cnxn.cursor()

#select_query = "SELECT * FROM [GTFSWithTraffic]"

#crsr.execute(select_query)


#data = crsr.fetchall()

#print(data)


In [29]:
df = pd.read_sql_query("SELECT * from [GTFSWithTraffic]", cnxn)

In [65]:
df["bus_Number"] = df["bus_Number"].astype(str)

In [66]:
df.dtypes

now                     datetime64[ns]
bus_Number                      object
I_O                             object
stop_sequence_no                 int64
stop_id                         object
stop_lat                       float64
stop_lng                       float64
Arr_Delay                      float64
curr_speed                       int64
freeflow_speed                   int64
curr_travel_time                 int64
freeflow_travel_time             int64
pressure                       float64
cloud                            int64
rain                             int64
humidity                       float64
hours                            int64
rush_hour                        int64
delay_yn                         int64
dtype: object

In [31]:
df["now"] = pd.to_datetime(df["now"])        

In [32]:
df["Arr_Delay"]

0          0.0
1       -120.0
2          0.0
3          0.0
4          0.0
         ...  
30493      0.0
30494      0.0
30495      0.0
30496      0.0
30497    240.0
Name: Arr_Delay, Length: 30498, dtype: float64

In [33]:
df["now"].iloc[1].hour

15

In [34]:
df['hours'] = np.nan
df["rush_hour"] = np.nan
    
for i in range(df.shape[0]):
    df['hours'].iloc[i] = df["now"].iloc[i].hour      

df['hours']  = df['hours'].astype("int64")

df["rush_hour"] = np.logical_or(df["hours"]== 7.0, df["hours"]== 8.0)
df["rush_hour"] = np.logical_or(df["rush_hour"] == True, df["hours"]== 16.0)
df["rush_hour"] = np.logical_or(df["rush_hour"] == True, df["hours"]== 17.0)
df["rush_hour"].replace({False:0,True:1},inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [35]:
df["delay_yn"] = [1 if x > 0 else 0 for x in df['Arr_Delay']]

In [36]:
df[df["delay_yn"]==1]

,now,bus_Number,I_O,stop_sequence_no,stop_id,stop_lat,stop_lng,Arr_Delay,curr_speed,freeflow_speed,curr_travel_time,freeflow_travel_time,pressure,cloud,rain,humidity,hours,rush_hour,delay_yn
6,2021-08-12 15:53:38,11,I,30,8220DB000886,53.322597,-6.248685,480.0,34,34,22,22,1012.0,0,0,75.0,15,0,1
10,2021-08-12 15:53:38,114,I,15,8250DB003179,0.000000,0.000000,60.0,0,0,0,0,1012.0,0,0,75.0,15,0,1
14,2021-08-12 15:53:38,120,I,15,8230DB002239,53.356168,-6.391727,360.0,58,75,12,9,1012.0,0,0,75.0,15,0,1
16,2021-08-12 15:53:38,122,I,24,8220DB001352,53.334445,-6.265407,120.0,23,23,23,23,1012.0,0,0,75.0,15,0,1
18,2021-08-12 15:53:38,123,I,25,8220DB001940,53.343539,-6.285734,180.0,24,43,91,51,1012.0,0,0,75.0,15,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30466,2021-08-18 11:47:50,7,O,30,8250DB003036,53.295844,-6.167656,120.0,45,45,9,9,1006.0,0,0,82.0,11,0,1
30486,2021-08-18 11:47:50,83,I,32,8220DB001617,53.349331,-6.274807,60.0,5,24,184,38,1006.0,0,0,82.0,11,0,1
30487,2021-08-18 11:47:50,83,O,36,8220DB001478,53.345646,-6.272943,60.0,4,20,446,89,1006.0,0,0,82.0,11,0,1
30490,2021-08-18 11:47:50,84,I,42,8350DB004303,53.187324,-6.099754,120.0,40,40,32,32,1006.0,0,0,82.0,11,0,1


In [41]:
df["bus_Number"].tolist()

['1',
 '1',
 '102',
 '102',
 '104',
 '104',
 '11',
 '11',
 '111',
 '111',
 '114',
 '114',
 '116',
 '118',
 '120',
 '120',
 '122',
 '122',
 '123',
 '123',
 '13',
 '13',
 '130',
 '130',
 '14',
 '14',
 '140',
 '140',
 '142',
 '142',
 '145',
 '145',
 '15',
 '15',
 '150',
 '150',
 '151',
 '151',
 '15A',
 '15A',
 '15B',
 '15B',
 '15D',
 '15D',
 '16',
 '16',
 '161',
 '161',
 '16D',
 '17',
 '17',
 '18',
 '18',
 '184',
 '184',
 '185',
 '185',
 '220',
 '220',
 '236',
 '236',
 '238',
 '238',
 '239',
 '239',
 '25',
 '25',
 '25A',
 '25A',
 '25B',
 '25B',
 '25D',
 '25D',
 '25X',
 '25X',
 '26',
 '26',
 '27',
 '27',
 '270',
 '270',
 '27A',
 '27A',
 '27B',
 '27B',
 '27X',
 '27X',
 '32',
 '32',
 '32X',
 '32X',
 '33',
 '33',
 '33D',
 '33D',
 '33E',
 '33X',
 '33X',
 '37',
 '37',
 '38',
 '38',
 '38A',
 '38A',
 '38B',
 '38B',
 '38D',
 '38D',
 '39',
 '39',
 '39A',
 '39A',
 '39X',
 '39X',
 '4',
 '4',
 '40',
 '40',
 '40B',
 '40B',
 '40D',
 '40D',
 '40E',
 '40E',
 '41',
 '41',
 '41B',
 '41B',
 '41C',
 '41C',
 '

In [ ]:
for i 

In [37]:
df.corr()[["delay_yn"]]

,delay_yn
stop_sequence_no,0.089305
stop_lat,-0.064553
stop_lng,0.063823
Arr_Delay,0.370248
curr_speed,-0.062740
freeflow_speed,-0.056327
curr_travel_time,-0.005286
freeflow_travel_time,-0.022291
pressure,0.003651
cloud,-0.005963


In [12]:
features = ["rain","humidity","pressure","cloud","curr_travel_time","freeflow_travel_time","curr_speed","freeflow_speed","hours","rush_hour"]

In [13]:
#x_train, x_test, y_train, y_test = train_test_split(df_total[features], df_total["significant_delay"], test_size=0.3, random_state=0)
x_train, x_test, y_train, y_test = train_test_split(df[features], df["delay_yn"], test_size=0.3, random_state=0, stratify=df["delay_yn"])

In [14]:
logreg = (LogisticRegression().fit(x_train[features], y_train))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [15]:
#train predictions
logreg_predictions_train = (logreg.predict(x_train[features])>= 0.5) * 1.0

In [16]:
#results on full training data set
#logreg_predictions_train = (logreg.predict(x_train[features]) >= 0.5) * 1.0

print("==================== Train Data =======================")
print("Overall Performance:")
print("Accuracy: {:2f}".format(metrics.accuracy_score(y_train, logreg_predictions_train)))
print("Precision: {:2f}".format(metrics.precision_score(y_train, logreg_predictions_train)))
print("f1: {:2f}".format(metrics.f1_score(y_train, logreg_predictions_train)))
print("Recall: {:2f}".format(metrics.recall_score(y_train,logreg_predictions_train)))   
print("\nConfusion matrix:")
print("\t\tLate Predicted")
print("Details\t\tNo\tYes")
print("Late (No)\t{}\t{}".format(metrics.confusion_matrix(y_train, logreg_predictions_train)[0][0],metrics.confusion_matrix(y_train, logreg_predictions_train)[0][1]))
print("Late (Yes)\t{}\t{}".format(metrics.confusion_matrix(y_train, logreg_predictions_train)[1][0],metrics.confusion_matrix(y_train, logreg_predictions_train)[1][1]))

print("\nClassification report:\n ", metrics.classification_report(y_train, logreg_predictions_train))
print("======================================================")

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


==================== Train Data =======================
Overall Performance:
Accuracy: 0.957795
Precision: 0.000000
f1: 0.000000
Recall: 0.000000

Confusion matrix:
		Late Predicted
Details		No	Yes
Late (No)	20447	0
Late (Yes)	901	0

Classification report:
                precision    recall  f1-score   support

           0       0.96      1.00      0.98     20447
           1       0.00      0.00      0.00       901

    accuracy                           0.96     21348
   macro avg       0.48      0.50      0.49     21348
weighted avg       0.92      0.96      0.94     21348



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
#test predictions
logreg_predictions_test = (logreg.predict(x_test[features])>= 0.5) * 1.0

In [18]:
#evaluation metrics (test)
print("\n==================== Test Data ======================")
print("Overall Performance:")
print("Accuracy: {:2f}".format(metrics.accuracy_score(y_test, logreg_predictions_test)))
print("Precision: {:2f}".format(metrics.precision_score(y_test, logreg_predictions_test)))
print("f1: {:2f}".format(metrics.f1_score(y_test, logreg_predictions_test)))
print("Recall: {:2f}".format(metrics.recall_score(y_test, logreg_predictions_test)))   
print("\nConfusion matrix:")
print("\t\tLate Predicted")
print("Details\t\tNo\tYes")
print("Late (No)\t{}\t{}".format(metrics.confusion_matrix(y_test, logreg_predictions_test)[0][0],metrics.confusion_matrix(y_test, logreg_predictions_test)[0][1]))
print("Late (Yes)\t{}\t{}".format(metrics.confusion_matrix(y_test, logreg_predictions_test)[1][0],metrics.confusion_matrix(y_test, logreg_predictions_test)[1][1]))

print("\nClassification report:\n ", metrics.classification_report(y_test, logreg_predictions_test))
print("======================================================")


==================== Test Data ======================
Overall Performance:
Accuracy: 0.957814
Precision: 0.000000
f1: 0.000000
Recall: 0.000000

Confusion matrix:
		Late Predicted
Details		No	Yes
Late (No)	8764	0
Late (Yes)	386	0

Classification report:
                precision    recall  f1-score   support

           0       0.96      1.00      0.98      8764
           1       0.00      0.00      0.00       386

    accuracy                           0.96      9150
   macro avg       0.48      0.50      0.49      9150
weighted avg       0.92      0.96      0.94      9150



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.p

# Decision Tree

In [56]:
routes

['104',
 '17',
 '32X',
 '79A',
 '122',
 '13',
 '67X',
 '40E',
 '83',
 '26',
 '41',
 '46E',
 '1',
 '25A',
 '66B',
 '33D',
 '15A',
 '4',
 '15D',
 '83A',
 '75',
 '41D',
 '77A',
 '270',
 '68X',
 '123',
 '15',
 '238',
 '239',
 '11',
 '142',
 '38',
 '37',
 '39',
 '54A',
 '41C',
 '41B',
 '40',
 '120',
 '185',
 '40B',
 '7',
 '7A',
 '16D',
 '56A',
 '76',
 '111',
 '27X',
 '59',
 '25',
 '39A',
 '61',
 '65',
 '67',
 '51D',
 '84',
 '184',
 '16',
 '102',
 '44B',
 '18',
 '33',
 '66',
 '49',
 '38B',
 '66A',
 '79',
 '25D',
 '140',
 '70',
 '53',
 '27B',
 '46A',
 '14',
 '41X',
 '68',
 '27',
 '161',
 '25X',
 '38A',
 '9',
 '7B',
 '38D',
 '66X',
 '150',
 '44',
 '43',
 '33E',
 '77X',
 '114',
 '145',
 '15B',
 '27A',
 '63',
 '47',
 '33X',
 '236',
 '130',
 '32',
 '151',
 '42',
 '25B',
 '65B',
 '7D',
 '84A',
 '84X',
 '40D',
 '68A',
 '220',
 '118',
 '39X',
 '69',
 '116',
 '69X']

In [58]:
features = ["rain","humidity","pressure","cloud","curr_travel_time","freeflow_travel_time","curr_speed","freeflow_speed","hours","rush_hour"]

In [59]:
#x_train, x_test, y_train, y_test = train_test_split(df_total[features], df_total["significant_delay"], test_size=0.3, random_state=0)
x_train, x_test, y_train, y_test = train_test_split(df[features], df["delay_yn"], test_size=0.3, random_state=0, stratify=df["delay_yn"])

In [ ]:
models = {}

In [ ]:
for i in routes[0:3]

In [49]:
rfc = RandomForestClassifier(n_estimators=100, max_features='auto', oob_score=True, random_state=0)
rfc.fit(x_train[features], y_train)

RandomForestClassifier(oob_score=True, random_state=0)

In [20]:
# build random forests on training set 
dtc10 = DecisionTreeClassifier(max_depth=10, random_state=0) 
dtc10.fit(x_train[features], y_train) 

print("Max depth 10: \n",dtc10)

Max depth 10: 
 DecisionTreeClassifier(max_depth=10, random_state=0)


In [21]:
dtc10 = DecisionTreeClassifier(max_depth=10, random_state=0)

rfc_predictions_train = rfc.predict(x_train[features])
x_train["prediction_rfc"] = rfc_predictions_train

In [22]:
rfc_predictions_train = rfc.predict(x_train[features])

In [23]:
#full train set
print("==================== Train Data =======================")
print("Overall Performance:")
print("Accuracy: ", metrics.accuracy_score(y_train, rfc_predictions_train))
print("Precision: {:2f}".format(metrics.precision_score(y_train, rfc_predictions_train)))
print("f1: {:2f}".format(metrics.f1_score(y_train, rfc_predictions_train)))
print("Recall: {:2f}\n".format(metrics.recall_score(y_train, rfc_predictions_train)))      

print("Late (No)\t{}\t{}".format(metrics.confusion_matrix(y_train, rfc_predictions_train)[0][0],metrics.confusion_matrix(y_train, rfc_predictions_train)[0][1]))
print("Late (Yes)\t{}\t{}".format(metrics.confusion_matrix(y_train, rfc_predictions_train)[1][0],metrics.confusion_matrix(y_train, rfc_predictions_train)[1][1]))
print("Classification report:\n ", metrics.classification_report(y_train, rfc_predictions_train))

==================== Train Data =======================
Overall Performance:
Accuracy:  0.9859003185310099
Precision: 0.953172
f1: 0.807422
Recall: 0.700333

Late (No)	20416	31
Late (Yes)	270	631
Classification report:
                precision    recall  f1-score   support

           0       0.99      1.00      0.99     20447
           1       0.95      0.70      0.81       901

    accuracy                           0.99     21348
   macro avg       0.97      0.85      0.90     21348
weighted avg       0.99      0.99      0.98     21348



In [24]:
rfc_predictions_test = rfc.predict(x_test[features])

In [25]:
#print test results
print("==================== Test Data =======================")
print("Accuracy: ", metrics.accuracy_score(y_test, rfc_predictions_test))
print("Precision: {:2f}".format(metrics.precision_score(y_test, rfc_predictions_test)))
print("f1: {:2f}".format(metrics.f1_score(y_test, rfc_predictions_test)))
print("Recall: {:2f}\n".format(metrics.recall_score(y_test, rfc_predictions_test)))

print("Late (No)\t{}\t{}".format(metrics.confusion_matrix(y_test, rfc_predictions_test)[0][0],metrics.confusion_matrix(y_test, rfc_predictions_test)[0][1]))
print("Late (Yes)\t{}\t{}".format(metrics.confusion_matrix(y_test, rfc_predictions_test)[1][0],metrics.confusion_matrix(y_test, rfc_predictions_test)[1][1]))
print("Classification report:\n ", metrics.classification_report(y_test, rfc_predictions_test))

==================== Test Data =======================
Accuracy:  0.952568306010929
Precision: 0.342105
f1: 0.193309
Recall: 0.134715

Late (No)	8664	100
Late (Yes)	334	52
Classification report:
                precision    recall  f1-score   support

           0       0.96      0.99      0.98      8764
           1       0.34      0.13      0.19       386

    accuracy                           0.95      9150
   macro avg       0.65      0.56      0.58      9150
weighted avg       0.94      0.95      0.94      9150



In [26]:
with open('models_secondary.pkl', 'wb') as handle:
        pickle.dump(rfc, handle, pickle.HIGHEST_PROTOCOL)

In [98]:
def getSecondaryodel(route):
    df_route = df[df["bus_Number"]==route]
    
    #x_train, x_test, y_train, y_test = train_test_split(df_total[features], df_total["significant_delay"], test_size=0.3, random_state=0)
#     x_train, x_test, y_train, y_test = train_test_split(df_route[features], df_route["delay_yn"], test_size=0.3, random_state=0, stratify=df_route["delay_yn"])
    
#     rfc = RandomForestClassifier(n_estimators=100, max_features='auto', oob_score=True, random_state=0)
#     rfc.fit(x_train[features], y_train)
    
#     # build random forests on training set 
#     dtc10 = DecisionTreeClassifier(max_depth=10, random_state=0) 
#     dtc10.fit(x_train[features], y_train) 

#     print("Max depth 10: \n",dtc10)
    
#     dtc10 = DecisionTreeClassifier(max_depth=10, random_state=0)

#     rfc_predictions_train = rfc.predict(x_train[features])
#     x_train["prediction_rfc"] = rfc_predictions_train
    
#     rfc_predictions_train = rfc.predict(x_train[features])
    
#     #full train set
#     print("==================== Train Data =======================")
#     print("Overall Performance:")
#     print("Accuracy: ", metrics.accuracy_score(y_train, rfc_predictions_train))
#     print("Precision: {:2f}".format(metrics.precision_score(y_train, rfc_predictions_train)))
#     print("f1: {:2f}".format(metrics.f1_score(y_train, rfc_predictions_train)))
#     print("Recall: {:2f}\n".format(metrics.recall_score(y_train, rfc_predictions_train)))      

#     print("Late (No)\t{}\t{}".format(metrics.confusion_matrix(y_train, rfc_predictions_train)[0][0],metrics.confusion_matrix(y_train, rfc_predictions_train)[0][1]))
#     print("Late (Yes)\t{}\t{}".format(metrics.confusion_matrix(y_train, rfc_predictions_train)[1][0],metrics.confusion_matrix(y_train, rfc_predictions_train)[1][1]))
#     print("Classification report:\n ", metrics.classification_report(y_train, rfc_predictions_train))
    
    
#     rfc_predictions_test = rfc.predict(x_test[features])
#     #print test results
#     print("==================== Test Data =======================")
#     print("Accuracy: ", metrics.accuracy_score(y_test, rfc_predictions_test))
#     print("Precision: {:2f}".format(metrics.precision_score(y_test, rfc_predictions_test)))
#     print("f1: {:2f}".format(metrics.f1_score(y_test, rfc_predictions_test)))
#     print("Recall: {:2f}\n".format(metrics.recall_score(y_test, rfc_predictions_test)))

#     print("Late (No)\t{}\t{}".format(metrics.confusion_matrix(y_test, rfc_predictions_test)[0][0],metrics.confusion_matrix(y_test, rfc_predictions_test)[0][1]))
#     print("Late (Yes)\t{}\t{}".format(metrics.confusion_matrix(y_test, rfc_predictions_test)[1][0],metrics.confusion_matrix(y_test, rfc_predictions_test)[1][1]))
#     print("Classification report:\n ", metrics.classification_report(y_test, rfc_predictions_test))
    
    final_model = DecisionTreeClassifier(max_depth=10, random_state=0) 
    final_model.fit(df_route[features], df_route["delay_yn"]) 
    
    return final_model

In [99]:
routes = list(set(df["bus_Number"].tolist()))

In [100]:
test = ["1","4"]

In [101]:
x = []

In [102]:
for i in test:
    x.append(getSecondaryodel(i))

In [81]:
x[0].predict()

DecisionTreeClassifier(max_depth=10, random_state=0)

In [91]:
asdc = df[df["bus_Number"]=="1"]

In [92]:
x[0].predict(asdc[features])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0], dtype=int64)

In [95]:
secondaryModels = {}

In [103]:
for i in routes:
    secondaryModels[i] = getSecondaryodel(i)

In [104]:
secondaryModels

{'104': DecisionTreeClassifier(max_depth=10, random_state=0),
 '17': DecisionTreeClassifier(max_depth=10, random_state=0),
 '32X': DecisionTreeClassifier(max_depth=10, random_state=0),
 '79A': DecisionTreeClassifier(max_depth=10, random_state=0),
 '122': DecisionTreeClassifier(max_depth=10, random_state=0),
 '13': DecisionTreeClassifier(max_depth=10, random_state=0),
 '67X': DecisionTreeClassifier(max_depth=10, random_state=0),
 '40E': DecisionTreeClassifier(max_depth=10, random_state=0),
 '83': DecisionTreeClassifier(max_depth=10, random_state=0),
 '26': DecisionTreeClassifier(max_depth=10, random_state=0),
 '41': DecisionTreeClassifier(max_depth=10, random_state=0),
 '46E': DecisionTreeClassifier(max_depth=10, random_state=0),
 '1': DecisionTreeClassifier(max_depth=10, random_state=0),
 '25A': DecisionTreeClassifier(max_depth=10, random_state=0),
 '66B': DecisionTreeClassifier(max_depth=10, random_state=0),
 '33D': DecisionTreeClassifier(max_depth=10, random_state=0),
 '15A': Decisio

In [105]:
with open('models_secondary.pkl', 'wb') as handle:
        pickle.dump(secondaryModels, handle, pickle.HIGHEST_PROTOCOL)

In [110]:
with open('models_secondary.pkl', 'rb') as f:
    secondaryModels = pickle.load(f)

In [111]:
secondaryModels

{'104': DecisionTreeClassifier(max_depth=10, random_state=0),
 '17': DecisionTreeClassifier(max_depth=10, random_state=0),
 '32X': DecisionTreeClassifier(max_depth=10, random_state=0),
 '79A': DecisionTreeClassifier(max_depth=10, random_state=0),
 '122': DecisionTreeClassifier(max_depth=10, random_state=0),
 '13': DecisionTreeClassifier(max_depth=10, random_state=0),
 '67X': DecisionTreeClassifier(max_depth=10, random_state=0),
 '40E': DecisionTreeClassifier(max_depth=10, random_state=0),
 '83': DecisionTreeClassifier(max_depth=10, random_state=0),
 '26': DecisionTreeClassifier(max_depth=10, random_state=0),
 '41': DecisionTreeClassifier(max_depth=10, random_state=0),
 '46E': DecisionTreeClassifier(max_depth=10, random_state=0),
 '1': DecisionTreeClassifier(max_depth=10, random_state=0),
 '25A': DecisionTreeClassifier(max_depth=10, random_state=0),
 '66B': DecisionTreeClassifier(max_depth=10, random_state=0),
 '33D': DecisionTreeClassifier(max_depth=10, random_state=0),
 '15A': Decisio